In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pulp import *
from copy import deepcopy
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser
from src.preprocessing.preprocessor import Preprocessor
from src.solvers.solution import Solution
from src.solvers.greedy import GreedySolver

from tqdm import tqdm
from typing import Dict

/tmp/ipykernel_307587/79102685.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [55]:
parser = Parser("data/testfiles/test5.txt")

In [56]:
info = parser.read()

1204it [00:00, 234412.20it/s]

In [57]:
dataset : Dict[int, pd.DataFrame ]= info['data']
p, K, M, N = info['p'], info['K'], info['M'], info['N']

In [58]:
preprocessor = Preprocessor(
    K, 
    M,
    N,
    p,
    dataset
)

data_1 = preprocessor.remove_trivial_values()
data_2 = preprocessor.remove_ip_dominated(dataset)
data_3 = preprocessor.remove_lp_dominated(data_2)

100%|██████████| 40/40 [00:00<00:00, 7604.92it/s]


100%|██████████| 40/40 [00:00<00:00, 493.86it/s]


136


In [61]:
solver = GreedySolver(
    K, 
    M,
    N,
    p,
    data_3
)

In [60]:
s = 0
for v in data_3.values():
    s+= len(v)

s

193

In [62]:
solver.solve(data_3)

In [63]:
solver.solution.get_remaining_power_budget()

0.0

In [64]:
solver.solution.get_data_rate()

1637.0

## Greedy algorithm

In [102]:
s

196

In [103]:
solution = Solution(
    K,
    M, 
    N, 
    p
)

for n in range(N):
    p_kmn = data_3[n]\
        .iloc[0]\
        ['p_k,m,n']
    
    r_kmn = data_3[n]\
        .iloc[0]\
        ['r_k,m,n']
    
    k = data_3[n]\
        .iloc[0]\
        ['k']
    
    m = data_3[n]\
        .iloc[0]\
        ['m']
    
    solution.consume_power_budget(p_kmn) 
    solution.add_data_rate(r_kmn)
    
    solution.set_triple(k,m,n,p_kmn, r_kmn)



In [104]:
all_pairs = []
for n in range(N):
    data_3[n]['inc_eff'] = (data_3[n]['r_k,m,n'] - data_3[n].shift(1)['r_k,m,n'])/(data_3[n]['p_k,m,n'] - data_3[n].shift(1)['p_k,m,n'])
    all_pairs.append(data_3[n].dropna())

all_pairs = pd.concat(all_pairs).reset_index(drop=True)

sorted_by_eff = all_pairs.sort_values(by = "inc_eff", ascending=False)

In [105]:
def popleft(df : pd.DataFrame):
    x = df.iloc[0]
    k = int(x.k)
    m = int(x.m)
    n = int(x.n)
    p_kmn = x['p_k,m,n']
    r_kmn = x['r_k,m,n']
    df = df.drop(index=x.name)

    return k,m,n, p_kmn, r_kmn, df

In [106]:
sorted_by_eff

,k,m,n,"p_k,m,n","r_k,m,n",inc_eff
109,2,0,29,22.0,35.0,27.000000
152,14,0,39,5.0,52.0,16.000000
74,5,0,19,15.0,24.0,11.500000
129,7,0,34,5.0,34.0,9.000000
54,10,0,15,54.0,29.0,5.250000
...,...,...,...,...,...,...
113,14,3,29,656.0,94.0,0.009828
101,11,3,26,537.0,96.0,0.008671
73,11,3,18,608.0,100.0,0.007075
97,9,3,25,593.0,100.0,0.002874


In [107]:
# greedy iteration

best_k, best_m, best_n, best_p_kmn, best_r_kmn, sorted_by_eff = popleft(sorted_by_eff.dropna())

k,m, p_kmn, r_kmn, name = solution.get_solution_at_channel(best_n)

while best_p_kmn - p_kmn <= solution.get_remaining_power_budget():

    solution.set_triple(
        int(best_k), 
        int(best_m),
        best_n,
        best_p_kmn,
        best_r_kmn
    )
    
    solution.consume_power_budget(best_p_kmn - p_kmn)

    solution.add_data_rate(best_r_kmn - r_kmn)
    
    if len(sorted_by_eff) > 0:
        best_k, best_m, best_n, best_p_kmn, best_r_kmn, sorted_by_eff = popleft(sorted_by_eff)
        k,m, p_kmn, r_kmn, name = solution.get_solution_at_channel(best_n)
    
    else: 
        break


In [108]:
solution.get_data_rate()

1637.0

## LP Solver

In [14]:
prob = LpProblem("UserScheduling", LpMaximize)

vars = []
R = []
P = []

every_channel_allocated_constraint = []

for n in data_3.keys():
    X_n = []
    for idx, row in data_3[n].iterrows():
        k = row['k']
        m = row['m']
        p_kmn = row['p_k,m,n']
        r_kmn = row['r_k,m,n']

        x = LpVariable(f"x_{int(k)},{int(m)},{int(n)}", 0, 1, None)
        
        X_n.append(x)
        vars.append(x)
        R.append(r_kmn)
        P.append(p_kmn)
    
    every_channel_allocated_constraint.append(
        lpSum(X_n) == 1
    )




In [15]:
objective = lpSum([R[i] * vars[i] for i in range(len(vars))])
constraint_power_budget = lpSum([P[i] * vars[i] for i in range(len(vars))])

prob += objective, "objective"
prob += constraint_power_budget <= p, "power_constraint"
for constraint in every_channel_allocated_constraint:
    prob += constraint

In [16]:
res = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/pedro/anaconda3/envs/user-scheduling-in-5g/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b4dc70ec98654694b9955e97cfddb1b3-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b4dc70ec98654694b9955e97cfddb1b3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 731 RHS
At line 773 BOUNDS
At line 1002 ENDATA
Problem MODEL has 41 rows, 228 columns and 456 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 41 (0) rows, 228 (0) columns and 456 (0) elements
0  Obj -0 Primal inf 63.376608 (40) Dual inf 13568.834 (228)
33  Obj 1637
Optimal - objective value 1637
Optimal objective 1637 - 33 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [17]:
for v in prob.variables():
    if v.varValue == 0:
        continue
    print(v.name , " = ", v.varValue)

x_0,0,27  =  1.0
x_0,1,14  =  1.0
x_0,1,19  =  1.0
x_1,0,0  =  1.0
x_1,0,1  =  1.0
x_1,0,8  =  1.0
x_1,0,9  =  1.0
x_1,2,10  =  1.0
x_10,0,12  =  1.0
x_10,0,15  =  1.0
x_10,0,18  =  1.0
x_10,0,28  =  1.0
x_10,0,33  =  1.0
x_10,0,5  =  1.0
x_11,0,30  =  1.0
x_11,0,31  =  1.0
x_11,3,37  =  1.0
x_13,0,13  =  1.0
x_13,0,17  =  1.0
x_13,0,35  =  1.0
x_13,0,4  =  1.0
x_14,0,39  =  1.0
x_14,0,7  =  1.0
x_14,1,26  =  1.0
x_2,0,11  =  1.0
x_2,0,24  =  1.0
x_2,0,29  =  1.0
x_2,1,16  =  1.0
x_3,0,2  =  1.0
x_3,0,20  =  1.0
x_4,0,22  =  1.0
x_4,0,32  =  1.0
x_5,0,21  =  1.0
x_5,0,23  =  1.0
x_6,0,38  =  1.0
x_7,0,36  =  1.0
x_7,0,6  =  1.0
x_7,1,34  =  1.0
x_8,1,3  =  1.0
x_9,0,25  =  1.0


In [265]:
prob.objective.value()

1637.0